In [1]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-6exx890d
Created temporary directory: /tmp/pip-req-tracker-fp6dy2q4
Created requirements tracker '/tmp/pip-req-tracker-fp6dy2q4'
Created temporary directory: /tmp/pip-install-hub1g1kw
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-xi7_j_d6
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-fp6dy2q4'
    Running setup.py (path:/tmp/pip-req-build-xi7_j_d6/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-xi7_j_d6/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-xi7_j_d6/pip-egg-info/apex.egg-info/PKG-INFO
    writing dependency_links to /tmp/pip-req-build-xi7_j_d6/pip-egg-info/apex.egg-info/dependency_links.txt
    writing top-level names to /tmp/

In [2]:
cd ..

/content


In [3]:
!pip install -U PyYAML

Requirement already up-to-date: PyYAML in /usr/local/lib/python3.7/dist-packages (5.4.1)


In [7]:
!git clone -b feature/DR-images https://github.com/jmarrietar/suncet.git

fatal: destination path 'suncet' already exists and is not an empty directory.


In [4]:
cd suncet

/content/suncet


In [7]:
!mkdir datasets

In [8]:
!mkdir datasets/dr

In [9]:
!mkdir logs

In [10]:
!python download.py -d sample@1000

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/suncet/datasets/dr/sample@1000.zip
108MB [00:00, 110MB/s] 


In [ ]:
!python3 main.py --sel paws_train --fname configs/paws/dr_train.yaml

INFO:root:called-params paws_train configs/paws/dr_train.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_epochs': 10,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 0,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
            

TO DO: Averiguar esto del Multicrop y por que PAWS funciono funciono luego de que lo cambiara a zero?. 

In [5]:
###################################
########### DEBUG #################
###################################

In [6]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_simclr_loss,
    init_suncet_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

In [7]:
import argparse

import torch.multiprocessing as mp

import pprint
import yaml

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [9]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [10]:
fname = args.fname
sel = args.sel

In [11]:
import logging
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [12]:
rank = 0

In [13]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    params = yaml.load(y_file, Loader=yaml.FullLoader)
    logger.info('loaded params...')
    if rank == 0:
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(params)

if rank == 0:
    dump = os.path.join(params['logging']['folder'], f'params-{sel}.yaml')
    with open(dump, 'w') as f:
        yaml.dump(params, f)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_epochs': 10,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
                'model_

In [14]:
args = params

# PAWS TRAIN

In [15]:
def init_model(
    device,
    model_name='resnet50',
    use_pred=False,
    output_dim=128
):
    if 'wide_resnet' in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]()
        hidden_dim = 2048
        if 'w2' in model_name:
            hidden_dim *= 2
        elif 'w4' in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(OrderedDict([
        ('fc1', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn1', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu1', torch.nn.ReLU(inplace=True)),
        ('fc2', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn2', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu2', torch.nn.ReLU(inplace=True)),
        ('fc3', torch.nn.Linear(hidden_dim, output_dim))
    ]))

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head['bn1'] = torch.nn.BatchNorm1d(output_dim)
        pred_head['fc1'] = torch.nn.Linear(output_dim, output_dim//mx)
        pred_head['bn2'] = torch.nn.BatchNorm1d(output_dim//mx)
        pred_head['relu'] = torch.nn.ReLU(inplace=True)
        pred_head['fc2'] = torch.nn.Linear(output_dim//mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder

In [16]:
def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0
):
    param_groups = [
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' not in n) and ('bn' not in n))},
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' in n) or ('bn' in n)),
         'LARS_exclude': True,
         'weight_decay': 0}
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr)
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup*iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs*iterations_per_epoch)
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler


In [17]:
import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_paws_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms,
    make_multicrop_transform
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

# --
log_timings = True
log_freq = 10
checkpoint_freq = 50
# --

_GLOBAL_SEED = 0
np.random.seed(_GLOBAL_SEED)
torch.manual_seed(_GLOBAL_SEED)
torch.backends.cudnn.benchmark = True

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

In [18]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META
model_name = args['meta']['model_name']
output_dim = args['meta']['output_dim']
load_model = args['meta']['load_checkpoint']
r_file = args['meta']['read_checkpoint']
copy_data = args['meta']['copy_data']
use_fp16 = args['meta']['use_fp16']
use_pred_head = args['meta']['use_pred_head']
device = torch.device(args['meta']['device'])
torch.cuda.set_device(device)

# -- CRITERTION
reg = args['criterion']['me_max']
supervised_views = args['criterion']['supervised_views']
classes_per_batch = args['criterion']['classes_per_batch']
s_batch_size = args['criterion']['supervised_imgs_per_class']
u_batch_size = args['criterion']['unsupervised_batch_size']
temperature = args['criterion']['temperature']
sharpen = args['criterion']['sharpen']

# -- DATA
unlabeled_frac = args['data']['unlabeled_frac']
color_jitter = args['data']['color_jitter_strength']
normalize = args['data']['normalize']
root_path = args['data']['root_path']
image_folder = args['data']['image_folder']
dataset_name = args['data']['dataset']
subset_path = args['data']['subset_path']
unique_classes = args['data']['unique_classes_per_rank']
multicrop = args['data']['multicrop']
label_smoothing = args['data']['label_smoothing']
data_seed = None
if 'cifar10' in dataset_name:
    data_seed = args['data']['data_seed']
    crop_scale = (0.75, 1.0) if multicrop > 0 else (0.5, 1.0)
    mc_scale = (0.3, 0.75)
    mc_size = 18
else:
    crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
    mc_scale = (0.05, 0.14)
    mc_size = 96

In [19]:
# -- OPTIMIZATION
wd = float(args['optimization']['weight_decay'])
num_epochs = args['optimization']['epochs']
warmup = args['optimization']['warmup']
start_lr = args['optimization']['start_lr']
lr = args['optimization']['lr']
final_lr = args['optimization']['final_lr']
mom = args['optimization']['momentum']
nesterov = args['optimization']['nesterov']

# -- LOGGING
folder = args['logging']['folder']
tag = args['logging']['write_tag']

In [20]:
label_smoothing

0.1

In [21]:
world_size = 1
rank = 0 

In [22]:
# -- log/checkpointing paths
log_file = os.path.join(folder, f'{tag}_r{rank}.csv')
save_path = os.path.join(folder, f'{tag}' + '-ep{epoch}.pth.tar')
latest_path = os.path.join(folder, f'{tag}-latest.pth.tar')
best_path = os.path.join(folder, f'{tag}' + '-best.pth.tar')
load_path = None
if load_model:
    load_path = os.path.join(folder, r_file) if r_file is not None else latest_path

In [23]:
# -- make csv_logger
csv_logger = CSVLogger(log_file,
                        ('%d', 'epoch'),
                        ('%d', 'itr'),
                        ('%.5f', 'paws-xent-loss'),
                        ('%.5f', 'paws-me_max-reg'),
                        ('%d', 'time (ms)'))

In [24]:
# -- init model
encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim)
if world_size > 1:
    process_group = apex.parallel.create_syncbn_process_group(0)
    encoder = apex.parallel.convert_syncbn_model(encoder, process_group=process_group)

In [25]:
# -- init losses
paws = init_paws_loss(
    #multicrop=multicrop,
    tau=temperature,
    T=sharpen,
    me_max=reg)

In [26]:
# -- assume support images are sampled with ClassStratifiedSampler
labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=world_size,
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing)

In [27]:
print("normalize {}".format(normalize))

normalize True


In [28]:
# -- make data transforms
transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize)
multicrop_transform = (multicrop, None)
if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
            dataset_name=dataset_name,
            num_crops=multicrop,
            size=mc_size,
            crop_scale=mc_scale,
            normalize=normalize,
            color_distortion=color_jitter)

In [29]:
# -- init data-loaders/samplers
(unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler) = init_data(
        dataset_name=dataset_name,
        transform=transform,
        init_transform=init_transform,
        supervised_views=supervised_views,
        u_batch_size=u_batch_size,
        s_batch_size=s_batch_size,
        unique_classes=unique_classes,
        classes_per_batch=classes_per_batch,
        multicrop_transform=multicrop_transform,
        world_size=world_size,
        rank=rank,
        root_path=root_path,
        image_folder=image_folder,
        training=True,
        copy_data=copy_data)
iter_supervised = None
ipe = len(unsupervised_loader)
logger.info(f'iterations per epoch: {ipe}')

self.multicrop_transform (0, None)
self.multicrop_transform (0, None)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
# -- init optimizer and scheduler
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
    encoder=encoder,
    weight_decay=wd,
    start_lr=start_lr,
    ref_lr=lr,
    final_lr=final_lr,
    ref_mom=mom,
    nesterov=nesterov,
    iterations_per_epoch=ipe,
    warmup=warmup,
    num_epochs=num_epochs)
if world_size > 1:
    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)


In [31]:
start_epoch = 0
# -- load training checkpoint
if load_model:
    encoder, optimizer, start_epoch = load_checkpoint(
        r_path=load_path,
        encoder=encoder,
        opt=optimizer,
        scaler=scaler,
        use_fp16=use_fp16)
    for _ in range(start_epoch):
        for _ in range(ipe):
            scheduler.step()

In [ ]:
################## DEBUG #################

In [32]:
epoch = 0 

In [33]:
# -- update distributed-data-loader epoch
unsupervised_sampler.set_epoch(epoch)
if supervised_sampler is not None:
    supervised_sampler.set_epoch(epoch)

loss_meter = AverageMeter()
ploss_meter = AverageMeter()
rloss_meter = AverageMeter()
time_meter = AverageMeter()
data_meter = AverageMeter()

In [34]:
best_loss = None

In [35]:
for itr, udata in enumerate(unsupervised_loader):

    def load_imgs():
        # -- unsupervised imgs
        uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
        # -- supervised imgs
        global iter_supervised
        try:
            sdata = next(iter_supervised)
        except Exception:
            iter_supervised = iter(supervised_loader)
            logger.info(f'len.supervised_loader: {len(iter_supervised)}')
            sdata = next(iter_supervised)
        finally:
            labels = torch.cat([labels_matrix for _ in range(supervised_views)])
            simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
        # -- concatenate supervised imgs and unsupervised imgs
        imgs = simgs + uimgs
        return imgs, labels
    (imgs, labels), dtime = gpu_timer(load_imgs)
    data_meter.update(dtime)
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [36]:
from logging import getLogger
import torch
from src.utils import (
    AllGather,
    AllReduce
)

In [43]:
multicrop=0
tau=0.1
T=0.25
me_max=True

In [44]:
with torch.cuda.amp.autocast(enabled=use_fp16):
    optimizer.zero_grad()

    # --
    # h: representations of 'imgs' before head
    # z: representations of 'imgs' after head
    # -- If use_pred_head=False, then encoder.pred (prediction
    #    head) is None, and _forward_head just returns the
    #    identity, z=h
    h, z = encoder(imgs, return_before_head=True)

    # Compute paws loss in full precision
    with torch.cuda.amp.autocast(enabled=False):

        # Step 1. convert representations to fp32
        h, z = h.float(), z.float()

        # Step 2. determine anchor views/supports and their
        #         corresponding target views/supports
        # --
        num_support = supervised_views * s_batch_size * classes_per_batch
        # --
        anchor_supports = z[:num_support]
        anchor_views = z[num_support:]
        # --
        target_supports = h[:num_support].detach()
        target_views = h[num_support:].detach()
        target_views = torch.cat([
            target_views[u_batch_size:2*u_batch_size],
            target_views[:u_batch_size]], dim=0)

        softmax = torch.nn.Softmax(dim=1)

        def sharpen(p):
            sharp_p = p**(1./T)
            sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
            return sharp_p

        def snn(query, supports, labels):
            """ Soft Nearest Neighbours similarity classifier """
            # Step 1: normalize embeddings
            query = torch.nn.functional.normalize(query)
            supports = torch.nn.functional.normalize(supports)

            # Step 2: gather embeddings from all workers
            supports = AllGather.apply(supports)

            # Step 3: compute similarlity between local embeddings
            return softmax(query @ supports.T / tau) @ labels


        #anchor_views=anchor_views
        #anchor_supports=anchor_supports
        anchor_support_labels=labels
        #target_views=target_views
        #target_supports=target_supports
        target_support_labels=labels
        # -- NOTE: num views of each unlabeled instance = 2+multicrop
        batch_size = len(anchor_views) // (2+multicrop)

        # Step 1: compute anchor predictions
        probs = snn(anchor_views, anchor_supports, anchor_support_labels)


        # Step 2: compute targets for anchor predictions
        with torch.no_grad():
            targets = snn(target_views, target_supports, target_support_labels)
            targets = sharpen(targets)
            if multicrop > 0:
                mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
                targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
            targets[targets < 1e-4] *= 0  # numerical stability

        # Step 3: compute cross-entropy loss H(targets, queries)
        loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

        # Step 4: compute me-max regularizer
        rloss = 0.
        if me_max:
            avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
            rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))




In [46]:
loss

tensor(0.6944, device='cuda:0', grad_fn=<MeanBackward0>)

In [41]:
batch_size

8

# Suncet Train

In [48]:
import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_paws_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms,
    make_multicrop_transform
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

# --
log_timings = True
log_freq = 10
checkpoint_freq = 50
# --

_GLOBAL_SEED = 0
np.random.seed(_GLOBAL_SEED)
torch.manual_seed(_GLOBAL_SEED)
torch.backends.cudnn.benchmark = True

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

In [17]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META
model_name = args['meta']['model_name']
output_dim = args['meta']['output_dim']
load_model = args['meta']['load_checkpoint']
r_file = args['meta']['read_checkpoint']
copy_data = args['meta']['copy_data']
use_fp16 = args['meta']['use_fp16']
use_pred_head = args['meta']['use_pred_head']
device = torch.device(args['meta']['device'])
torch.cuda.set_device(device)

# -- CRITERTION
supervised_epochs = args['criterion']['supervised_epochs']
supervised_views = args['criterion']['supervised_views']
classes_per_batch = args['criterion']['classes_per_batch']
s_batch_size = args['criterion']['supervised_imgs_per_class']
u_batch_size = args['criterion']['unsupervised_batch_size']
temperature = args['criterion']['temperature']

# -- DATA
unlabeled_frac = args['data']['unlabeled_frac']
color_jitter = args['data']['color_jitter_strength']
normalize = args['data']['normalize']
root_path = args['data']['root_path']
image_folder = args['data']['image_folder']
dataset_name = args['data']['dataset']
subset_path = args['data']['subset_path']
unique_classes = args['data']['unique_classes_per_rank']
label_smoothing = args['data']['label_smoothing']
crop_scale = (0.5, 1.0) if 'cifar10' in dataset_name else (0.08, 1.0)
data_seed = None
if 'cifar10' in dataset_name:
    data_seed = args['data']['data_seed']

# -- OPTIMIZATION
wd = float(args['optimization']['weight_decay'])
num_epochs = args['optimization']['epochs']
warmup = args['optimization']['warmup']
start_lr = args['optimization']['start_lr']
lr = args['optimization']['lr']
final_lr = args['optimization']['final_lr']
mom = args['optimization']['momentum']
nesterov = args['optimization']['nesterov']

# -- LOGGING
folder = args['logging']['folder']
tag = args['logging']['write_tag']

In [18]:
from suncet.src.suncet_train import init_model, init_opt

In [51]:
# -- log/checkpointing paths
log_file = os.path.join(folder, f'{tag}_r{rank}.csv')
save_path = os.path.join(folder, f'{tag}' + '-ep{epoch}.pth.tar')
latest_path = os.path.join(folder, f'{tag}-latest.pth.tar')
best_path = os.path.join(folder, f'{tag}' + '-best.pth.tar')
load_path = None
if load_model:
    load_path = os.path.join(folder, r_file) if r_file is not None else latest_path

In [52]:
# -- make csv_logger
csv_logger = CSVLogger(log_file,
                        ('%d', 'epoch'),
                        ('%d', 'itr'),
                        ('%.5f', 'paws-xent-loss'),
                        ('%.5f', 'paws-me_max-reg'),
                        ('%d', 'time (ms)'))

In [19]:
encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim)

In [20]:
world_size = 1

In [21]:
ntxent = init_simclr_loss(
    batch_size=u_batch_size,
    world_size=world_size,
    rank=rank,
    temperature=temperature,
    device=device)

In [22]:
suncet = init_suncet_loss(
        num_classes=classes_per_batch,
        batch_size=s_batch_size*supervised_views,
        world_size=world_size,
        rank=rank,
        temperature=temperature,
        device=device,
        unique_classes=unique_classes)

In [23]:
# -- assume support images are sampled with ClassStratifiedSampler
labels = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=world_size,
    device=device,
    #unique_classes=unique_classes,
    unique_classes=False,
    smoothing=label_smoothing)

In [24]:
labels.shape

torch.Size([64, 2])

In [25]:
unique_classes

False

In [26]:
# -- make data transforms
transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize)

In [27]:
_GLOBAL_SEED = 0

In [28]:
########### DEBUG INTERIOR FUNCTION ##########

def _init_dr_data(
    transform,
    init_transform,
    u_batch_size,
    s_batch_size,
    classes_per_batch,
    unique_classes=False,
    multicrop_transform=(0, None),
    supervised_views=1,
    world_size=1,
    rank=0,
    root_path='/datasets/',
    image_folder='imagenet_full_size/061417/',
    training=True,
    copy_data=False,
    tar_folder='imagenet_full_size/',
    tar_file='imagenet_full_size-061417.tar'
):
    imagedr = ImageDR(
        root=root_path,
        image_folder=image_folder,
        tar_folder=tar_folder,
        tar_file=tar_file,
        transform=transform,
        train=training,
        copy_data=copy_data)
    logger.info('ImageDR dataset created')
    unsupervised_set = TransImageDR(
        dataset=imagedr,
        supervised=False,
        init_transform=init_transform,
        multicrop_transform=multicrop_transform,
        seed=_GLOBAL_SEED)
    unsupervised_sampler = torch.utils.data.distributed.DistributedSampler(
        dataset=unsupervised_set,
        num_replicas=world_size,
        rank=rank)
    unsupervised_loader = torch.utils.data.DataLoader(
        unsupervised_set,
        sampler=unsupervised_sampler,
        batch_size=u_batch_size,
        drop_last=True,
        pin_memory=True,
        num_workers=8)
    logger.info('ImageDR unsupervised data loader created')

    supervised_sampler, supervised_loader = None, None
    if classes_per_batch > 0 and s_batch_size > 0:
        logger.info('Making supervised ImageDR data loader...')
        supervised_set = TransImageDR(
            dataset=imagedr,
            supervised=True,
            supervised_views=supervised_views,
            init_transform=init_transform,
            seed=_GLOBAL_SEED)
        supervised_sampler = ClassStratifiedSampler(
            data_source=supervised_set,
            world_size=world_size,
            rank=rank,
            batch_size=s_batch_size,
            classes_per_batch=classes_per_batch,
            unique_classes=unique_classes,
            seed=_GLOBAL_SEED)
        supervised_loader = torch.utils.data.DataLoader(
            supervised_set,
            batch_sampler=supervised_sampler,
            pin_memory=True,
            num_workers=8)
        if len(supervised_loader) > 0:
            tmp = ceil(len(unsupervised_loader) / len(supervised_loader))
            supervised_sampler.set_inner_epochs(tmp)
            logger.info(f'supervised-reset-period {tmp}')
        logger.info('ImageDR supervised data loader created')

    return (unsupervised_loader, unsupervised_sampler,
            supervised_loader, supervised_sampler)

In [29]:
tar_folder='imagenet_full_size/',
tar_file='imagenet_full_size-061417.tar'
training=True
copy_data=False

In [30]:
class ClassStratifiedSampler(torch.utils.data.Sampler):

    def __init__(
        self,
        data_source,
        world_size,
        rank,
        batch_size=1,
        classes_per_batch=10,
        epochs=1,
        seed=0,
        unique_classes=False
    ):
        """
        ClassStratifiedSampler

        Batch-sampler that samples 'batch-size' images from subset of randomly
        chosen classes e.g., if classes a,b,c are randomly sampled,
        the sampler returns
            torch.cat([a,b,c], [a,b,c], ..., [a,b,c], dim=0)
        where a,b,c, are images from classes a,b,c respectively.
        Sampler, samples images WITH REPLACEMENT (i.e., not epoch-based)

        :param data_source: dataset of type "TransImageNet" or "TransCIFAR10'
        :param world_size: total number of workers in network
        :param rank: local rank in network
        :param batch_size: num. images to load from each class
        :param classes_per_batch: num. classes to randomly sample for batch
        :param epochs: num consecutive epochs thru data_source before gen.reset
        :param seed: common seed across workers for subsampling classes
        :param unique_classes: true ==> each worker samples a distinct set of classes; false ==> all workers sample the same classes
        """
        super(ClassStratifiedSampler, self).__init__(data_source)
        self.data_source = data_source

        self.rank = rank
        self.world_size = world_size
        self.cpb = classes_per_batch
        self.unique_cpb = unique_classes
        self.batch_size = batch_size
        self.num_classes = len(data_source.classes)
        self.epochs = epochs
        self.outer_epoch = 0

        if not self.unique_cpb:
            assert self.num_classes % self.cpb == 0

        self.base_seed = seed  # instance seed
        self.seed = seed  # subsample sampler seed

    def set_epoch(self, epoch):
        self.outer_epoch = epoch

    def set_inner_epochs(self, epochs):
        self.epochs = epochs

    def _next_perm(self):
        self.seed += 1
        g = torch.Generator()
        g.manual_seed(self.seed)
        self._perm = torch.randperm(self.num_classes, generator=g)

    def _get_perm_ssi(self):
        start = self._ssi
        end = self._ssi + self.cpb
        subsample = self._perm[start:end]
        return subsample

    def _next_ssi(self):
        if not self.unique_cpb:
            self._ssi = (self._ssi + self.cpb) % self.num_classes
            if self._ssi == 0:
                self._next_perm()
        else:
            self._ssi += self.cpb * self.world_size
            max_end = self._ssi + self.cpb * (self.world_size - self.rank)
            if max_end > self.num_classes:
                self._ssi = self.rank * self.cpb
                self._next_perm()

    def _get_local_samplers(self, epoch):
        """ Generate samplers for local data set in given epoch """
        seed = int(self.base_seed + epoch
                   + self.epochs * self.rank
                   + self.outer_epoch * self.epochs * self.world_size)
        g = torch.Generator()
        g.manual_seed(seed)
        samplers = []
        for t in range(self.num_classes):
            t_indices = np.array(self.data_source.target_indices[t])
            if not self.unique_cpb:
                i_size = len(t_indices) // self.world_size
                if i_size > 0:
                    t_indices = t_indices[self.rank*i_size:(self.rank+1)*i_size]
            if len(t_indices) > 1:
                t_indices = t_indices[torch.randperm(len(t_indices), generator=g)]
            samplers.append(iter(t_indices))
        return samplers

    def _subsample_samplers(self, samplers):
        """ Subsample a small set of samplers from all class-samplers """
        subsample = self._get_perm_ssi()
        subsampled_samplers = []
        for i in subsample:
            subsampled_samplers.append(samplers[i])
        self._next_ssi()
        return zip(*subsampled_samplers)

    def __iter__(self):
        self._ssi = self.rank*self.cpb if self.unique_cpb else 0
        self._next_perm()

        # -- iterations per epoch (extract batch-size samples from each class)
        ipe = (self.num_classes // self.cpb if not self.unique_cpb
               else self.num_classes // (self.cpb * self.world_size)) * self.batch_size

        for epoch in range(self.epochs):

            # -- shuffle class order
            samplers = self._get_local_samplers(epoch)
            subsampled_samplers = self._subsample_samplers(samplers)

            counter, batch = 0, []
            for i in range(ipe):
                batch += list(next(subsampled_samplers))
                counter += 1
                if counter == self.batch_size:
                    yield batch
                    counter, batch = 0, []
                    if i + 1 < ipe:
                        subsampled_samplers = self._subsample_samplers(samplers)

    def __len__(self):
        if self.batch_size == 0:
            return 0

        ipe = (self.num_classes // self.cpb if not self.unique_cpb
               else self.num_classes // (self.cpb * self.world_size))
        return self.epochs * ipe



In [31]:
from src.data_manager import ImageDR, TransImageDR
# ClassStratifiedSampler

In [32]:
imagedr = ImageDR(
    root=root_path,
    image_folder=image_folder,
    tar_folder=tar_folder,
    tar_file=tar_file,
    transform=transform,
    train=training,
    copy_data=copy_data)

In [33]:
multicrop_transform=(0, None)

In [34]:
unsupervised_set = TransImageDR(
    dataset=imagedr,
    supervised=False,
    init_transform=init_transform,
    multicrop_transform=multicrop_transform,
    seed=_GLOBAL_SEED)
unsupervised_sampler = torch.utils.data.distributed.DistributedSampler(
    dataset=unsupervised_set,
    num_replicas=world_size,
    rank=rank)
unsupervised_loader = torch.utils.data.DataLoader(
    unsupervised_set,
    sampler=unsupervised_sampler,
    batch_size=u_batch_size,
    drop_last=True,
    pin_memory=True,
    num_workers=8)

self.multicrop_transform (0, None)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
supervised_set = TransImageDR(
    dataset=imagedr,
    supervised=True,
    supervised_views=supervised_views,
    init_transform=init_transform,
    seed=_GLOBAL_SEED)

self.multicrop_transform (0, None)


In [36]:
supervised_sampler = ClassStratifiedSampler(
    data_source=supervised_set,
    world_size=world_size,
    #world_size=4,
    rank=rank,
    batch_size=s_batch_size,
    #classes_per_batch=classes_per_batch,
    classes_per_batch=2,
    #unique_classes=unique_classes,
    unique_classes=False,
    seed=_GLOBAL_SEED)

In [37]:
len(supervised_sampler)

1

In [38]:
supervised_loader = torch.utils.data.DataLoader(
    supervised_set,
    batch_sampler=supervised_sampler,
    pin_memory=True,
    num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [39]:
len(supervised_loader)

1

In [40]:
"""
# -- init data-loaders/samplers
(unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler) = init_data(
        dataset_name=dataset_name,
        transform=transform,
        init_transform=init_transform,
        supervised_views=supervised_views,
        u_batch_size=u_batch_size,
        s_batch_size=s_batch_size,
        unique_classes=unique_classes,
        classes_per_batch=classes_per_batch,
        world_size=world_size,
        rank=rank,
        root_path=root_path,
        image_folder=image_folder,
        training=True,
        copy_data=copy_data)
iter_supervised = None
ipe = len(unsupervised_loader)
logger.info(f'iterations per epoch: {ipe}')
"""

"\n# -- init data-loaders/samplers\n(unsupervised_loader,\n    unsupervised_sampler,\n    supervised_loader,\n    supervised_sampler) = init_data(\n        dataset_name=dataset_name,\n        transform=transform,\n        init_transform=init_transform,\n        supervised_views=supervised_views,\n        u_batch_size=u_batch_size,\n        s_batch_size=s_batch_size,\n        unique_classes=unique_classes,\n        classes_per_batch=classes_per_batch,\n        world_size=world_size,\n        rank=rank,\n        root_path=root_path,\n        image_folder=image_folder,\n        training=True,\n        copy_data=copy_data)\niter_supervised = None\nipe = len(unsupervised_loader)\nlogger.info(f'iterations per epoch: {ipe}')\n"

In [41]:
ipe = len(unsupervised_loader)
print(f'iterations per epoch: {ipe}')

iterations per epoch: 31


In [42]:
# -- init optimizer and scheduler
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
    encoder=encoder,
    weight_decay=wd,
    start_lr=start_lr,
    ref_lr=lr,
    final_lr=final_lr,
    ref_mom=mom,
    nesterov=nesterov,
    iterations_per_epoch=ipe,
    warmup=warmup,
    num_epochs=num_epochs)

In [43]:
start_epoch = 0
# -- load training checkpoint
if load_model:
    encoder, optimizer, start_epoch = load_checkpoint(
        r_path=load_path,
        encoder=encoder,
        opt=optimizer,
        scaler=scaler,
        use_fp16=use_fp16)
    for _ in range(start_epoch):
        for _ in range(ipe):
            scheduler.step()

In [44]:
iter_supervised = iter(supervised_loader)
print(f'len.supervised_loader: {len(iter_supervised)}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


len.supervised_loader: 1


In [45]:
len(iter_supervised)

1

In [53]:
# -- TRAINING LOOP
best_loss = None
for epoch in range(start_epoch, num_epochs):
    logger.info('Epoch %d' % (epoch + 1))

    # -- update distributed-data-loader epoch
    unsupervised_sampler.set_epoch(epoch)
    if supervised_sampler is not None:
        supervised_sampler.set_epoch(epoch)

    loss_meter = AverageMeter()
    ploss_meter = AverageMeter()
    rloss_meter = AverageMeter()
    time_meter = AverageMeter()
    data_meter = AverageMeter()

    for itr, udata in enumerate(unsupervised_loader):
        if use_fp16:
            udata = [u.half() for u in udata]

        def load_imgs():
            # -- unsupervised imgs (2 views)
            imgs = [u.to(device) for u in udata[:2]]

            # -- labeled imgs
            global iter_supervised
            try:
                sdata = next(iter_supervised)
            except Exception:
                iter_supervised = iter(supervised_loader)
                logger.info(f'len.supervised_loader: {len(iter_supervised)}')
                sdata = next(iter_supervised)
                if use_fp16:
                    sdata = [s.half() for s in sdata]
            finally:
                # -- shuffle images in sampled support mini-batch
                simgs = [s.to(device) for s in sdata[:-1]]

            # -- concatenate unlabeled images and labeled images
            return torch.cat(imgs + simgs, dim=0)
        (imgs), dtime = gpu_timer(load_imgs)
        data_meter.update(dtime)

        def train_step():
            with torch.cuda.amp.autocast(enabled=use_fp16):
                optimizer.zero_grad()
                z = encoder(imgs)
                # -- simclr loss
                uloss = ntxent(z[:2*u_batch_size])
                # -- suncet loss
                sloss = suncet(z[2*u_batch_size:], labels)
                loss = uloss
                if epoch < supervised_epochs:
                    loss += sloss

            scaler.scale(loss).backward()
            lr_stats = scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            return (float(loss), float(sloss), float(uloss), lr_stats)
        (loss, ploss, rloss, lr_stats), etime = gpu_timer(train_step)
        loss_meter.update(loss)
        ploss_meter.update(ploss)
        rloss_meter.update(rloss)
        time_meter.update(etime)

        if (itr % log_freq == 0) or np.isnan(loss) or np.isinf(loss):
            csv_logger.log(epoch + 1, itr,
                            ploss_meter.avg,
                            rloss_meter.avg,
                            time_meter.avg)
            logger.info('[%d, %5d] loss: %.3f (%.3f %.3f) '
                        '(%d ms; %d ms)'
                        % (epoch + 1, itr,
                            loss_meter.avg,
                            ploss_meter.avg,
                            rloss_meter.avg,
                            time_meter.avg,
                            data_meter.avg))
            if lr_stats is not None:
                logger.info('[%d, %5d] lr_stats: %.3f (%.2e, %.2e)'
                            % (epoch + 1, itr,
                                lr_stats.avg,
                                lr_stats.min,
                                lr_stats.max))

        assert not np.isnan(loss), 'loss is nan'

    # -- logging/checkpointing
    logger.info('avg. loss %.3f' % loss_meter.avg)

    if rank == 0:
        save_dict = {
            'encoder': encoder.state_dict(),
            'opt': optimizer.state_dict(),
            'epoch': epoch + 1,
            'unlabel_prob': unlabeled_frac,
            'loss': loss_meter.avg,
            's_batch_size': s_batch_size,
            'u_batch_size': u_batch_size,
            'world_size': world_size,
            'lr': lr,
            'temperature': temperature,
            'amp': scaler.state_dict()
        }
        torch.save(save_dict, latest_path)
        if best_loss is None or best_loss > loss_meter.avg:
            best_loss = loss_meter.avg
            logger.info('updating "best" checkpoint')
            torch.save(save_dict, best_path)
        if (epoch + 1) % checkpoint_freq == 0 \
                or (epoch + 1) % 10 == 0 and epoch < checkpoint_freq:
            torch.save(save_dict, save_path.format(epoch=f'{epoch + 1}'))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

In [ ]:
########### DEBUG TRAIN STEP #################

In [51]:
# -- update distributed-data-loader epoch
unsupervised_sampler.set_epoch(epoch)
if supervised_sampler is not None:
    supervised_sampler.set_epoch(epoch)

In [52]:
for itr, udata in enumerate(unsupervised_loader):
    if use_fp16:
        udata = [u.half() for u in udata]

    def load_imgs():
        # -- unsupervised imgs (2 views)
        imgs = [u.to(device) for u in udata[:2]]

        # -- labeled imgs
        global iter_supervised
        try:
            sdata = next(iter_supervised)
        except Exception:
            iter_supervised = iter(supervised_loader)
            logger.info(f'len.supervised_loader: {len(iter_supervised)}')
            sdata = next(iter_supervised)
            if use_fp16:
                sdata = [s.half() for s in sdata]
        finally:
            # -- shuffle images in sampled support mini-batch
            simgs = [s.to(device) for s in sdata[:-1]]

        # -- concatenate unlabeled images and labeled images
        return torch.cat(imgs + simgs, dim=0)

    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [53]:
udata[0].shape

torch.Size([32, 3, 224, 224])

In [54]:
udata[1].shape

torch.Size([32, 3, 224, 224])

In [55]:
(imgs), dtime = gpu_timer(load_imgs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [56]:
imgs.shape

torch.Size([128, 3, 224, 224])

In [57]:
def init_suncet_loss(
    num_classes,
    batch_size,
    world_size,
    rank,
    temperature,
    device,
    unique_classes=False
):
    """
    Make SuNCEt supervised contrastive loss

    NOTE: Assumes data is loaded with data-loaders constrcuted from 'init_data'
          method in data_manager.py

    :param num_classes: num. image classes per batch
    :param batch_size: num. images per class in each batch
    :param world_size: total number of workers in network
    :param rank: local rank in network
    :param temperature: temp. param
    :param device: device to map tensors onto
    :param unique_classes: whether each worker loads the same set of classes
    """
    local_images = batch_size*num_classes
    total_images = local_images*world_size
    diag_mask = torch.ones(local_images, total_images).to(device)
    offset = rank*local_images
    for i in range(local_images):
        diag_mask[i, offset + i] = 0.

    def contrastive_loss(z, labels):

        # Step 1: normalize embeddings
        z = torch.nn.functional.normalize(z)

        # Step 2: gather embeddings from all workers
        z_buffer = AllGather.apply(z)

        # Step 3: compute class predictions
        exp_cs = torch.exp(z @ z_buffer.T / temperature) * diag_mask
        probs = exp_cs.div(exp_cs.sum(dim=1, keepdim=True)) @ labels

        # Step 4: compute loss for predictions
        targets = labels[offset:offset+local_images]
        overlap = probs**(-targets)
        loss = torch.mean(torch.sum(torch.log(overlap), dim=1))
        return loss

    return contrastive_loss

In [58]:
suncet = init_suncet_loss(
        num_classes=classes_per_batch,
        batch_size=s_batch_size*supervised_views,
        world_size=world_size,
        rank=rank,
        temperature=temperature,
        device=device,
        unique_classes=unique_classes)

In [58]:
with torch.cuda.amp.autocast(enabled=use_fp16):
    optimizer.zero_grad()
    z = encoder(imgs)
    # -- simclr loss
    uloss = ntxent(z[:2*u_batch_size])
    # -- suncet loss
    sloss = suncet(z[2*u_batch_size:], labels)
    loss = uloss

RuntimeError: ignored

In [59]:
z = z[2*u_batch_size:]

In [60]:
from src.utils import (
    AllGather,
    AllReduce
)

In [61]:
s_batch_size

32

In [62]:
with torch.cuda.amp.autocast(enabled=use_fp16):

    batch_size=s_batch_size*supervised_views
    num_classes=classes_per_batch

    local_images = batch_size*num_classes
    total_images = local_images*world_size
    diag_mask = torch.ones(local_images, total_images).to(device)
    offset = rank*local_images
    for i in range(local_images):
        diag_mask[i, offset + i] = 0.

    # Step 1: normalize embeddings
    z = torch.nn.functional.normalize(z)

    # Step 2: gather embeddings from all workers
    z_buffer = AllGather.apply(z)

    exp_cs = torch.exp(z @ z_buffer.T / temperature) * diag_mask

    # Step 3: compute class predictions
    probs = exp_cs.div(exp_cs.sum(dim=1, keepdim=True)) @ labels

RuntimeError: ignored

In [63]:
exp_cs.shape

NameError: ignored

In [76]:
labels.shape

torch.Size([16, 2])

In [ ]:






# Step 4: compute loss for predictions
targets = labels[offset:offset+local_images]
overlap = probs**(-targets)
loss = torch.mean(torch.sum(torch.log(overlap), dim=1))

In [ ]:
best_loss = None
for epoch in range(start_epoch, num_epochs):
    logger.info('Epoch %d' % (epoch + 1))

    # -- update distributed-data-loader epoch
    unsupervised_sampler.set_epoch(epoch)
    if supervised_sampler is not None:
        supervised_sampler.set_epoch(epoch)

    loss_meter = AverageMeter()
    ploss_meter = AverageMeter()
    rloss_meter = AverageMeter()
    time_meter = AverageMeter()
    data_meter = AverageMeter()

    for itr, udata in enumerate(unsupervised_loader):
        if use_fp16:
            udata = [u.half() for u in udata]

        def load_imgs():
            # -- unsupervised imgs (2 views)
            imgs = [u.to(device) for u in udata[:2]]

            # -- labeled imgs
            global iter_supervised
            try:
                sdata = next(iter_supervised)
            except Exception:
                iter_supervised = iter(supervised_loader)
                logger.info(f'len.supervised_loader: {len(iter_supervised)}')
                sdata = next(iter_supervised)
                if use_fp16:
                    sdata = [s.half() for s in sdata]
            finally:
                # -- shuffle images in sampled support mini-batch
                simgs = [s.to(device) for s in sdata[:-1]]

            # -- concatenate unlabeled images and labeled images
            return torch.cat(imgs + simgs, dim=0)
        (imgs), dtime = gpu_timer(load_imgs)
        data_meter.update(dtime)

        def train_step():
            with torch.cuda.amp.autocast(enabled=use_fp16):
                optimizer.zero_grad()
                z = encoder(imgs)
                # -- simclr loss
                uloss = ntxent(z[:2*u_batch_size])
                # -- suncet loss
                sloss = suncet(z[2*u_batch_size:], labels)
                loss = uloss
                if epoch < supervised_epochs:
                    loss += sloss


In [131]:
z[2*u_batch_size:].shape

torch.Size([32, 2048])

In [132]:
z.shape

torch.Size([96, 2048])

In [133]:
labels.shape

torch.Size([16, 2])

In [134]:
sloss = suncet(z[2*u_batch_size:], labels)

RuntimeError: ignored

In [ ]:
##############################################

In [1]:
!nvidia-smi

Fri Jun 18 02:36:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------